In [2]:
# Load packages for data analysis
import pandas as pd

# Load packages for Big Query 
from google.cloud import bigquery
import os

### Set-up

**Set-up: GCP interface** 

Run before querying to setup Big Query

In [3]:
# Define configurations for Big Query - Stride Datalake
project_id = 'som-nero-phi-boussard' # Location of stride datalake
db = "som-nero-phi-boussard.stride_datalake" # Define the database
msc = "som-nero-phi-boussard.MSc_ACU_Oncology"
es = "som-nero-phi-boussard.ES_ACU_Oncology"
client = bigquery.Client(project=project_id) # Set project to project_id
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/path/to/your/credentials.json'
os.environ['GCLOUD_PROJECT'] = "som-nero-phi-boussard" # specify environment
db = "som-nero-phi-boussard" # Define the database

/Users/eliasaquand/miniforge3/envs/env/lib/python3.9/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
# 1) Specify the job config to properly read the file
job_config = bigquery.LoadJobConfig()
job_config.autodetect = True # determines the datatype of the variable
job_config.write_disposition = 'WRITE_TRUNCATE'
job_config.max_bad_records = 1 # allow 5 bad records; 

# Read schema from JSON
# job_config.schema = self.bq_client.schema_from_json(
# f"{json_schema_dir}/{custom_mapping_table}.json")

# 2) Specify destination
# destination = f"som-nero-phi-boussard.MSc_ACU_Oncology.[COHORT NAME]"

# 3) Save file ob Big Query, using result from so far; client is specified above - implemented in the file 
# load_job = client.load_table_from_dataframe(dataframe = chemo_tx_dd,                                  
#                                                     destination = destination,
#                                                     job_config = job_config)

# Run the job:
# load_job.result()

### Load cohort

In [5]:
sql_query = f""" SELECT * FROM {es}.cohort_p1_text"""
cohort = (client.query(sql_query)).to_dataframe()

In [6]:
cohort

,PAT_DEID,HOSP_ADMSN_TIME,CHE_TX_DATE
0,1770802,2020-02-07 20:54:00,2020-01-16 00:00:00
1,3945576,2020-02-01 09:14:00,2020-01-24 00:00:00
2,1045640,2016-02-15 20:12:00,2016-02-08 00:00:00
3,1289918,2016-04-15 23:13:00,2016-03-16 00:00:00
4,2766580,2016-04-13 17:43:00,2016-03-24 00:00:00
...,...,...,...
1748,538184,2015-02-13 13:13:00,2015-01-16 08:59:00
1749,1253627,2019-03-14 16:38:00,2019-03-08 23:59:00
1750,3456627,2018-11-12 13:20:00,2018-10-17 09:59:00
1751,3316652,2014-06-06 19:31:00,2014-06-05 23:59:00


### Load the notes

In [7]:
# Load relevant note table
sql_query = f""" SELECT * FROM {es}.note_table"""
note_table = (client.query(sql_query)).to_dataframe()

In [8]:
note_table.isna().sum()

PAT_DEID          0
NOTE              0
ENCOUNTER_DATE    0
EFFECTIVE_TIME    0
NOTE_DESC         0
WORD_COUNT        0
dtype: int64

In [9]:
note_table.shape

(95536, 6)

In [12]:
note_table.NOTE_DESC.value_counts()[:15]

Progress Notes            33847
Care Plan Note            10517
ED Notes                   9466
Telephone Encounter        8341
Consults                   7225
Sign Out Note              3114
Consult Follow-Up          3070
H&P                        2451
Discharge Summary          1969
Discharge Instructions     1656
ED Provider Notes          1630
Patient Instructions       1587
Procedures                 1342
Assessment & Plan Note     1329
Advance Care Planning      1168
Name: NOTE_DESC, dtype: int64

In [9]:
len(pd.unique(note_table["PAT_DEID"]))

1753

In [11]:
note_table[note_table["NOTE_DESC"] == "ED Notes"].iloc[2].NOTE

'Straight catherization performed by this RN with Crystal, medstudent, assist.  Patient tolerated procedure well. UA sent to lab.   '

In [10]:
df_ed = note_table[note_table["NOTE_DESC"] == "ED Provider Notes"].iloc[:3]
df_hosp = note_table[note_table["NOTE_DESC"].isin(["H&P", "Discharge Summary"])].iloc[:5]

In [23]:
df_hosp

,PAT_DEID,NOTE,ENCOUNTER_DATE,EFFECTIVE_TIME,NOTE_DESC,WORD_COUNT
11,3692124,Stanford Hospital and Clinics History and Phy...,2021-05-06,2021-05-06 17:41:00,H&P,1872
59,46788,Stanford Hospital and Clinics Admission Histo...,2017-05-24,2017-05-24 19:57:00,H&P,2793
81,4112475,Stanford Hospital and Clinics Interventional ...,2020-08-20,2020-08-20 07:17:00,H&P,686
85,1337916,Stanford Hospital and Clinics Discharge Summa...,2016-08-18,2016-08-21 08:08:00,Discharge Summary,2404
119,2649310,Stanford Hospital and Clinics Discharge Summa...,2015-02-03,2015-02-05 13:10:00,Discharge Summary,2765


In [24]:
df_ed.to_csv('../df_ed.csv', index=False)
df_hosp.to_csv('../df_hosp.csv', index=False)

### Load the ACU event 

In [11]:
# Load all ACU events information of cohort patient 
sql_query = f""" SELECT * FROM {es}.cohort_p1_sym_op30"""
cohort_symp = (client.query(sql_query)).to_dataframe()

In [12]:
print(f'Number of patients: {len(pd.unique(cohort_symp.PAT_DEID))}')

Number of patients: 1753


In [13]:
cohort_symp_ed = cohort_symp[cohort_symp["PT_CLASS"] == "Emergency Services"]
cohort_symp_hosp = cohort_symp[cohort_symp["PT_CLASS"] == "Inpatient"]

In [14]:
idx_ed = pd.unique(cohort_symp_ed.PAT_DEID)
idx_hosp = pd.unique(cohort_symp_hosp.PAT_DEID)
len(idx_ed), len(idx_hosp)

(496, 1352)

In [15]:
idx_only_ed = list(set(idx_ed) - set(idx_hosp))
idx_only_hosp = list(set(idx_hosp) - set(idx_ed))
idx_ed_hosp = list(set(idx_ed).intersection(set(idx_hosp)))
len(idx_only_ed), len(idx_only_hosp), len(idx_ed_hosp)

(401, 1257, 95)

- Check if all patients with ED have an ED provider notes

In [16]:
df_ed = note_table[note_table["PAT_DEID"].isin(idx_ed)]
df_ed = df_ed[note_table["NOTE_DESC"] == "ED Provider Notes"]

<ipython-input-16-7f7d1462d159>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_ed = df_ed[note_table["NOTE_DESC"] == "ED Provider Notes"]


In [17]:
len(pd.unique(df_ed["PAT_DEID"]))

496

In [18]:
df_ed.shape

(679, 6)

- Check if all patients with hospitalization have an H&P notes

In [25]:
df_hosp = note_table[note_table["PAT_DEID"].isin(idx_hosp)]
df_hosp = df_hosp[note_table["NOTE_DESC"].isin(["H&P"])]

<ipython-input-25-2ec9d9af0e03>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_hosp = df_hosp[note_table["NOTE_DESC"].isin(["H&P"])]


In [26]:
len(pd.unique(df_hosp["PAT_DEID"]))

1319

In [27]:
df_hosp

,PAT_DEID,NOTE,ENCOUNTER_DATE,EFFECTIVE_TIME,NOTE_DESC,WORD_COUNT
11,3692124,Stanford Hospital and Clinics History and Phy...,2021-05-06,2021-05-06 17:41:00,H&P,1872
59,46788,Stanford Hospital and Clinics Admission Histo...,2017-05-24,2017-05-24 19:57:00,H&P,2793
135,1216368,Stanford Hospital and Clinics Interventional ...,2019-01-28,2019-01-28 10:15:00,H&P,1434
137,2778961,Stanford Hospital and Clinics Inpatient Hist...,2020-08-21,2020-08-21 22:55:00,H&P,2673
229,2666976,Stanford Hospital and Clinics Critical Care H...,2017-02-25,2017-02-25 20:28:00,H&P,2916
...,...,...,...,...,...,...
95307,962602,Stanford Hospital and Clinics Interventional ...,2014-04-07,2014-04-07 15:44:00,H&P,685
95316,2188221,Stanford Hospital and Clinics History and Phy...,2013-10-03,2013-10-03 19:33:00,H&P,1655
95503,400384,Stanford Hospital and Clinics History and Phy...,2016-08-10,2016-08-10 17:31:00,H&P,2361
95521,2649310,Stanford Hospital and Clinics Internal Medici...,2015-01-12,2015-01-12 14:56:00,H&P,2237


- Concatenate relevant note for ED patients and for Inpatient patients

In [34]:
note_by_type = pd.concat([df_ed, df_hosp])

In [35]:
note_by_type.shape

(5002, 6)

In [36]:
len(pd.unique(note_by_type["PAT_DEID"]))

1753

In [37]:
# Save note table
# Specify destination for storing dataframe
destination = f"som-nero-phi-boussard.ES_ACU_Oncology.note_by_type"

# Save file to Big Query
load_job = client.load_table_from_dataframe(dataframe = note_by_type,                                  
                                                   destination = destination,
                                                   job_config = job_config)

# Run the job:
load_job.result()

LoadJob<project=som-nero-phi-boussard, location=US, id=d3b2df32-e0cd-4cce-942d-a44566e8110f>

### Select relevant note types

In [10]:
# Note types 
#note_types = ["Progress Notes", "Telephone Encounter", "ED Notes", "Discharge Summary", "ED Provider Notes", "H&P", "Care Plan Note"]


In [11]:
#note_by_type = note_table.loc[note_table['NOTE_DESC'].isin(note_types)]

In [12]:
#len(pd.unique(note_by_type["PAT_DEID"]))

1753

In [ ]:
# Save note table
# Specify destination for storing dataframe
#destination = f"som-nero-phi-boussard.ES_ACU_Oncology.note_by_type"

# Save file to Big Query
#load_job = client.load_table_from_dataframe(dataframe = note_by_type,                                  
                                                   destination = destination,
                                                   job_config = job_config)

# Run the job:
#load_job.result()

### Analyze the different note types

In [349]:
total_counts = pd.DataFrame(note_table.NOTE_DESC.value_counts(normalize=True))
total_counts.rename(columns={"NOTE_DESC": "COUNT"}, inplace=True)
total_counts["NOTE_DESC"] = total_counts.index
total_counts.reset_index(drop=True, inplace=True)

In [351]:
total_counts[total_counts["COUNT"] >= 0.015].NOTE_DESC.values

array(['Progress Notes', 'Care Plan Note', 'ED Notes',
       'Telephone Encounter', 'Consults', 'Sign Out Note',
       'Consult Follow-Up', 'H&P', 'Discharge Summary',
       'Discharge Instructions', 'ED Provider Notes',
       'Patient Instructions'], dtype=object)

In [352]:
most_frequent_type = total_counts[total_counts["COUNT"] >= 0.015].NOTE_DESC.values

In [353]:
total_counts = pd.DataFrame(note_table.NOTE_DESC.value_counts())
total_counts.rename(columns={"NOTE_DESC": "COUNT"}, inplace=True)
total_counts["NOTE_DESC"] = total_counts.index
total_counts.reset_index(drop=True, inplace=True)

In [359]:
total_counts = total_counts[total_counts["NOTE_DESC"].isin(most_frequent_type)]

In [360]:
relevant_note = note_table['NOTE'].str.upper().apply(lambda x: any(y in x for y in all_classes))
relevant_note_table = note_table.loc[relevant_note]


In [361]:
relevant_counts = pd.DataFrame(relevant_note_table.NOTE_DESC.value_counts())
relevant_counts.rename(columns={"NOTE_DESC": "COUNT"}, inplace=True)
relevant_counts["NOTE_DESC"] = relevant_counts.index
relevant_counts.reset_index(drop=True, inplace=True)

In [362]:
d = pd.DataFrame()
note_desc = []
perc = []
for note_type in total_counts.NOTE_DESC.values:
    if note_type in relevant_counts["NOTE_DESC"].values:
        note_desc.append(note_type)
        perc.append(float((relevant_counts[relevant_counts["NOTE_DESC"] == note_type].COUNT) / float(total_counts[total_counts["NOTE_DESC"] == note_type].COUNT)) * 100)
         
d["NOTE_DESC"] = note_desc
d["PERC"] = perc

In [365]:
d.sort_values("PERC", ascending=False)

,NOTE_DESC,PERC
10,ED Provider Notes,99.149970
8,Discharge Summary,98.324873
7,H&P,95.434162
9,Discharge Instructions,90.365051
6,Consult Follow-Up,88.892508
5,Sign Out Note,79.749679
4,Consults,69.983393
0,Progress Notes,64.446346
11,Patient Instructions,56.819610
1,Care Plan Note,56.776278


In [363]:
d[d["PERC"] >= 50].sort_values("PERC", ascending=False)

,NOTE_DESC,PERC
10,ED Provider Notes,99.149970
8,Discharge Summary,98.324873
7,H&P,95.434162
9,Discharge Instructions,90.365051
6,Consult Follow-Up,88.892508
5,Sign Out Note,79.749679
4,Consults,69.983393
0,Progress Notes,64.446346
11,Patient Instructions,56.819610
1,Care Plan Note,56.776278


In [217]:
# Check 5-10 notes of each note type in the note_types list
note_table[note_table["NOTE_DESC"]== "Sign Out Note"].iloc[31].NOTE

"Justin Michael Kelly 9/6/2013 12:27 PM   PRIMARY TEAM: Treatment Team: Tt Med9 Np A, Pgr 27093  Patient's condition has unchanged since last report. Primary issue(s) this shift: CHEMO NEURO: AAO X3. MAE. Ambulates without difficulty CARDIAC: VSS. NO edema, no chest pains RESP: BS clear. No SOB. O2 sat 99-100 % on RA GI/GU: Abdomen soft + BS> No BM. Voids SKIN: no skin breakdown  Additional issue(s) this shift: CHEMO start. BEACON orders released Follow up items:  Chemo Core Data sheet-Advance Directive Complete? yes Advance Directive complete? no   IV access: yes L power port accessed. With good blood return Non-urgent issue for MD to address in next shift: none  Cruz, Maria Luz, RN   "

In [58]:
df = note_table.groupby(["NOTE_DESC"], as_index=False)["PAT_DEID"].count()

In [9]:
grouped = note_table.groupby(["PAT_DEID", "NOTE_DESC"], as_index=False)["PAT_DEID"].count()

In [28]:
grouped

,NOTE_DESC,PAT_DEID
0,"Consent Blood Transfusions, Blood Products, an...",1
1,Discharge Summary,1
2,H&P,1
3,Letter,1
4,Outpatient Letter,1
...,...,...
20216,H&P,2
20217,Patient Instructions,4
20218,Progress Notes,25
20219,Telephone Encounter,10


In [10]:
all_note_types = pd.unique(note_table.NOTE_DESC.values)

In [11]:
len(pd.unique(note_table["PAT_DEID"]))

1768

In [60]:
df_pat = pd.DataFrame()
note_desc = []
pat_count = []
# Get the number of patients that have each type of notes 
for note_type in all_note_types:
    note_desc.append(note_type)
    pat_count.append(len(pd.unique(note_table[note_table["NOTE_DESC"] == note_type]["PAT_DEID"])))
    #print(f'Number of patients with {note_type}: {len(pd.unique(grouped[grouped["NOTE_DESC"] == note_type]["PAT_DEID"]))}')

df_pat["NOTE_DESC"] = note_desc
df_pat["NUMBER_OF_PATIENTS"] = pat_count

In [61]:
df_pat.sort_values("NUMBER_OF_PATIENTS", ascending=False)[:20]

,NOTE_DESC,NUMBER_OF_PATIENTS
1,Progress Notes,1752
7,Telephone Encounter,1555
13,Discharge Summary,1378
6,H&P,1377
8,Consults,1310
2,ED Provider Notes,1303
0,ED Notes,1278
9,Discharge Instructions,1276
3,Patient Instructions,940
5,Care Plan Note,882


In [76]:
# Note types selected by Malvika 
note_types = ["Progress Notes", "Telephone Encounter", "ED Notes", "Discharge Summary", "ED Provider Notes", "H&P", "Care Plan Note"]

#==> consider Discharge Summary only if we use discharge codes for inpatient (TO CHECK)

In [77]:
note_by_type = note_table.loc[note_table['NOTE_DESC'].isin(note_types)]

In [78]:
len(pd.unique(note_by_type["PAT_DEID"]))

1768

In [79]:
len(pd.unique(note_table["PAT_DEID"]))

1768

In [80]:
# Save note table
# Specify destination for storing dataframe
destination = f"som-nero-phi-boussard.ES_ACU_Oncology.note_by_type"

# Save file to Big Query
load_job = client.load_table_from_dataframe(dataframe = note_by_type,                                  
                                                   destination = destination,
                                                   job_config = job_config)

# Run the job:
load_job.result()

LoadJob<project=som-nero-phi-boussard, location=US, id=4f997a1b-153b-40b2-889d-15896ceecad0>

In [27]:
len(pd.unique(note_table["PAT_DEID"]))

1768

In [70]:
df = note_table.NOTE_DESC.value_counts().to_frame()
df

,NOTE_DESC
Progress Notes,18931
Telephone Encounter,8115
Care Plan Note,4109
ED Notes,2856
Consults,2603
...,...
Outside Photographs,1
OUTSD NURSING NOTES,1
Transfer Summary,1
HP COMPLEM MED INTAKE FORM,1


In [71]:
df[df["NOTE_DESC"] > 1000]

,NOTE_DESC
Progress Notes,18931
Telephone Encounter,8115
Care Plan Note,4109
ED Notes,2856
Consults,2603
Sign Out Note,1674
Patient Instructions,1589
H&P,1129
Consult Follow-Up,1022


In [121]:
# Get the number of patients that have each type of notes 
for note_type in note_types:
    min_val = note_table[note_table["NOTE_DESC"]==note_type]["WORD_COUNT"].min()
    max_val = note_table[note_table["NOTE_DESC"]==note_type]["WORD_COUNT"].max()
    mean_val = note_table[note_table["NOTE_DESC"]==note_type]["WORD_COUNT"].mean()
    median_val = note_table[note_table["NOTE_DESC"]==note_type]["WORD_COUNT"].median()
    print(f'Stats for {note_type}: min = {min_val}; max: {max_val}; mean: {mean_val}; median: {median_val}')

Stats for Progress Notes: min = 3; max: 13596; mean: 1009.601975595584; median: 612.0
Stats for Telephone Encounter: min = 3; max: 4791; mean: 168.117067159581; median: 103.0
Stats for ED Notes: min = 3; max: 589; mean: 23.50455182072829; median: 14.0
Stats for Discharge Summary: min = 37; max: 7919; mean: 2521.6407538280328; median: 2471.0
Stats for ED Provider Notes: min = 17; max: 5734; mean: 2089.3762376237623; median: 2060.0
Stats for H&P: min = 10; max: 7913; mean: 2057.523472099203; median: 2049.0
Stats for Nursing Note: min = 3; max: 710; mean: 107.97014925373135; median: 42.0


In [106]:
# Obtain statistics for each note type in the note_types list
note_table[note_table["NOTE_DESC"]=="ED Provider Notes"]["WORD_COUNT"].median()

2060.0

In [69]:
# Check 5-10 notes of each note type in the note_types list
note_table[note_table["NOTE_DESC"]=="Care Plan Note"].iloc[100].NOTE

' Problem: Adult Inpatient Plan of Care Goal: Plan of Care Review Outcome: Ongoing, progressing Goal: Optimal Comfort and Wellbeing Outcome: Ongoing, progressing   Problem: Fall Injury Risk Goal: Absence of Fall and Fall-Related Injury Outcome: Ongoing, progressing  Patient is aware of plan of care. VSS. Using bedside commode to promote mobility. Defecating frequently. No complaints of pain. '

Insights: 
- Nursing Note are often pretty short and don't look useful (min = 3; max = 710; mean = 108; median = 42.0)
- ED Provider Notes are often pretty (min = 17; max = 5734; mean = 2089; median = 2060)


In [24]:
# Select the note types by Malvika & Tina
#note_types = ["Progress Notes", "Telephone Encounter", "ED Notes", "Discharge Summary", "ED Provider Notes", "H&P", "Nursing Note"]
# Start simple like Claudio (by Doug) according to Max
note_types = ["Progress Notes", "H&P"]

In [235]:
# Note types selected by Malvika 
note_types = ["Progress Notes", "Telephone Encounter", "ED Notes", "Discharge Summary", "ED Provider Notes", "H&P", "Care Plan Note", "Document Classification", "Consults", "Consult Follow-Up", "Discharge Instruction", "Patient Instructions", "RN Transfer Note", "ED Temp/RAP Patient"]

In [236]:
note_by_type = note_table.loc[note_table['NOTE_DESC'].isin(note_types)]

In [279]:
pat_note = note_by_type[note_by_type["PAT_DEID"] == 10789]
pat_note.NOTE_DESC.value_counts()

Progress Notes          21
Consults                 7
Telephone Encounter      5
H&P                      4
ED Notes                 3
ED Provider Notes        2
Consult Follow-Up        2
RN Transfer Note         2
Discharge Summary        2
Patient Instructions     1
Care Plan Note           1
Name: NOTE_DESC, dtype: int64

In [280]:
relevant_note = pat_note['NOTE'].str.upper().apply(lambda x: any(y in x for y in class_1 + emesis + dehydration))
relevant_note_table = pat_note.loc[relevant_note]


In [281]:
relevant_note_table.NOTE_DESC.value_counts()

Progress Notes          14
H&P                      4
Consults                 3
ED Provider Notes        2
RN Transfer Note         2
Discharge Summary        2
Telephone Encounter      2
Patient Instructions     1
Consult Follow-Up        1
Care Plan Note           1
Name: NOTE_DESC, dtype: int64

In [266]:
note_by_type[note_by_type["EFFECTIVE_TIME"] == "2018-02-17 09:32:00"].iloc[0].NOTE

'Stanford Hospital and Clinics Department of General Surgery Division of Surgical Oncology  Discharge Summary      Attending Physician: Brendan Visser, MD  Admission Date: 2/14/2018  Discharge Date: 2/17/2018    Admission Diagnosis: GASTRIC ADENOCARCINOMA  Discharge Diagnosis: Same as above  Operative Procedure: Procedure(s): IR BILIARY DRAIN REPLACEMENT    Reason for Hospitalization: Post-operative monitoring  History of Present Illness:  33 Y\xa0F\xa0h/o perforated duodenal ulcer s/p exploratory laparotomy and repair with Graham patch (10/14/17), gastric outlet obstruction s/p exploratory laparotomy, antrectomy, revision of prior gastrojejunostomy, Bilroth I gastroduodenoscopy with cholecystectomy, placement of common bile duct T-tube (12/28/17), and recently diagnosed T4aN2M1 gastric adenocarcinoma s/p cycle 1 of FOLFOX on 02/07/18. She was admitted urgently on Sunday 2/12 for worsening symptoms related to her gastric outlet obstruction. She underwent EGD and duodenal stent placemen

In [ ]:
# Check les patient que jai checker dabord et si pas assez de varite: check ce patient 10789 

In [27]:
#note_table[note_table["NOTE_DESC"]=="H&P"].iloc[50].NOTE

In [28]:
note_by_type

,PAT_DEID,NOTE,EFFECTIVE_TIME,NOTE_DESC,WORD_COUNT
2,400384,Stanford Hospital and Clinics History and Phy...,2016-08-10 17:31:00,H&P,2361
3,400384,Stanford Hospital and Clinics History and Phy...,2016-08-31 13:05:00,H&P,2031
4,2775296,Stanford Hospital and Clinics Pre-Procedure ...,2013-11-05 12:24:00,H&P,1598
5,1618176,Stanford Hospital and Clinics Interventional ...,2013-03-11 08:57:00,H&P,471
6,2360321,Stanford Hospital and Clinics History and P...,2015-02-06 18:12:00,H&P,2070
...,...,...,...,...,...
34182,2222591,Stanford Hospital and Clinics Gynecology Onco...,2017-10-09 08:30:00,Progress Notes,3014
34183,2222591,Patient's Name: Maria Y Diaz Today's Date: 10...,2017-10-17 11:43:00,Progress Notes,235
34184,2222591,Patient's Name: Maria Y Diaz Today's Date: 10...,2017-10-09 12:33:00,Progress Notes,216
34185,2222591,Stanford Health Care Palliative Medicine P...,2017-10-12 11:00:00,Progress Notes,2939


In [29]:
print(f'Number of patient with relevant types of notes: {len(pd.unique(note_by_type["PAT_DEID"]))}')

Number of patient with relevant types of notes: 1724


**We remove 35 patients**

TODO: check the removed patient notes

In [34]:
removed_patient = list(set(note_table.PAT_DEID).difference(set(note_by_type.PAT_DEID)))

In [35]:
len(removed_patient)

35

In [47]:
# TODO: look at the text of the removed patients
note_table[note_table["PAT_DEID"].isin(removed_patient)]

,PAT_DEID,NOTE,EFFECTIVE_TIME,NOTE_DESC,WORD_COUNT
1345,375611,STANFORD HOSPITAL AND CLINICS EMERGENCY DEPAR...,2013-02-14 00:00:00,Letter,321
1430,3727954,"Vascular Clinic and Lab 300 Pasteur Drive,...",2018-03-16 00:00:00,Letter,381
1540,2936693,STANFORD HOSPITAL AND CLINICS EMERGENCY DEPAR...,2014-05-10 00:00:00,Letter,277
1755,1742521,STANFORD HOSPITAL AND CLINICS EMERGENCY DEPAR...,2014-01-29 00:00:00,Letter,315
1883,616931,Jill and John Freidenrich Breast Center Stan...,2013-08-20 00:00:00,Letter,2131
...,...,...,...,...,...
48626,3122045,Interventional Radiology Patient Discharge E...,2019-06-17 17:03:00,Discharge Instructions,1290
48877,91863,You have pneumonia on the left side. You ...,2018-12-07 17:58:00,Discharge Instructions,57
49550,616931,,2013-08-22 07:28:00,Consent Dobutamine Stress Test,3
49873,1374054,Initial -H&P/Assessment obtained from Referrin...,2019-08-30 11:09:00,Transfer Center Initial Clinical Screen,622


In [30]:
# Save the cohort patient informations 
# Specify destination for storing dataframe
destination = f"som-nero-phi-boussard.ES_ACU_Oncology.note_by_type"

# Save file to Big Query
load_job = client.load_table_from_dataframe(dataframe = note_by_type,                                  
                                                   destination = destination,
                                                   job_config = job_config)

# Run the job:
load_job.result()

LoadJob<project=som-nero-phi-boussard, location=US, id=3bcc8d70-a515-4823-a231-6d081a1a9280>

### Check patients with few or too much notes

In [12]:
# Load relevant note table
sql_query = f""" SELECT * FROM {es}.note_by_type"""
note_by_type = (client.query(sql_query)).to_dataframe()

In [41]:
df = note_by_type.groupby(["PAT_DEID"]).count()

**Check patients with a few number of notes**

In [49]:
df_one = df[df.NOTE == 1]
df_one.shape

(106, 4)

=> 106 patients have only one notes

In [50]:
df_one

,NOTE,EFFECTIVE_TIME,NOTE_DESC,WORD_COUNT
PAT_DEID,,,,
38474,1,1,1,1
71347,1,1,1,1
87359,1,1,1,1
98007,1,1,1,1
132939,1,1,1,1
...,...,...,...,...
3822000,1,1,1,1
3866679,1,1,1,1
3901667,1,1,1,1


In [65]:
note_by_type[note_by_type["PAT_DEID"].isin(df_one.index)]

,PAT_DEID,NOTE,EFFECTIVE_TIME,NOTE_DESC,WORD_COUNT
74,3788812,Stanford Hospital and Clinics History and Phy...,2018-05-09 08:52:00,H&P,2907
106,1817618,Stanford Hospital and Clinics History and Phy...,2017-12-27 14:12:00,H&P,2445
147,219931,Stanford Hospital and Clinics Interventional ...,2014-05-02 12:33:00,H&P,393
161,1587486,Stanford Hospital and Clinics Interventional ...,2016-02-26 09:46:00,H&P,645
329,2304324,Stanford Hospital and Clinics Interventional ...,2020-01-13 06:55:00,H&P,1448
...,...,...,...,...,...
33072,598766,Patient came to ITA for Firmagon 240mg injec...,2018-05-09 13:48:00,Progress Notes,62
33135,3053550,RADIATION ONCOLOGY ON-TREATMENT NOTE Name: ...,2018-11-08 13:35:00,Progress Notes,901
33189,2376175,ITA SW covering note: SW briefly met pt and...,2013-03-15 15:38:00,Progress Notes,153
33428,2766580,Patient tolerated Rituxan infusion well. Vital...,2016-03-24 16:21:00,Progress Notes,36


**Check patients with too much notes**

In [60]:
df_max = df[df.NOTE == df.NOTE.max()]

In [61]:
df_max

,NOTE,EFFECTIVE_TIME,NOTE_DESC,WORD_COUNT
PAT_DEID,,,,
3177403,103,103,103,103


In [72]:
list(note_by_type[note_by_type["PAT_DEID"].isin(df_max.index)][:1].NOTE)

['Stanford Health Care   Occupational Therapy  Progress Note     Mitul Patel is a 27 Y male.  Room:B318    Admitting Diagnosis: CAUDA EQUINA COMPRESSION  Patient/Caregiver goals reviewed and integrated into rehab treatment plan?: Yes  Patient/Caregiver teaching performed?: Yes  Participants: Patient    Abbreviated Patient History: Cauda equina compression due to mets to spine     Precautions:  Precautions: Fall Risk Precautions, Spinal Precautions - No bending, no twisting, no lifting >5lbs    Subjective: "Pop it, poke a hole in it."  Pt did not like the firmness of the mattress when max-inflating the bed prior to therapy.           Pain Level:  (Pt reports pain. Pt stated he declined pain meds because he doesn\'t want to become addicted. )/10   Scale: Verbal 0-10   Location: Abdomen   Intervention: Repositioned, Rest, Ambulation/Increased Activity, RN Notified (Comments)     Reassessment: Patient in no acute distress    Coordinated care with PT.   Pt received in supine. NAD. Pt consen

**Check the length of the notes**

In [117]:
df = note_by_type[note_by_type["WORD_COUNT"] <= 12]

In [118]:
df.shape

(190, 5)

In [119]:
list(df.NOTE)

['Please see Gail Egan noted 9/5/2019    ',
 'NO SHOW    ',
 'Paged   ',
 'Patient left with transport for radiation therapy.    ',
 'Informed husband of the $1004.38 co-pay for Neupogen.   ',
 'Entered in Error.   ',
 'No Adherence call needed,Patient has appointment today at Oncology Clinic  ',
 'Lab drawn as ordered, no complication.   ',
 'Coram and Pathways updated of possible D/C Wednesday or Thursday.  ',
 'Error    ',
 'Wellness survey reviewed. No referrals needed.     ',
 'Associated Pathology Medical Group, Inc  Outside Path  ',
 'Faxed order/facesheet for hip abduction brace  ',
 'Erroneously opened encounter  ',
 'To MRI at 1845  ',
 'Charted in error  ',
 'Infusion completed at 0130.  ',
 '  ',
 'Saline mouth wash   ',
 'Pt refused arimedex, would like to spw team first.   ',
 'Oxygen tank delivered to pt for discharge  ',
 '1150ml of sang. Pleural fluid obtained during pleural cath placement  ',
 'Verbal report given to Gila RN on fground.  ',
 'Informed the team med 9 v

**we remove 460 patients**

In [95]:
irrelevant_note = note_by_type['NOTE'].str.upper().apply(lambda x: any(y in x for y in ["SOCIAL WORK PROGRESS NOTE"]))
irrelevant_note_table = note_by_type.loc[irrelevant_note]
relevant_note_table = note_by_type.loc[~irrelevant_note]

In [118]:
irrelevant_note_table.NOTE.iloc[1]

"SOCIAL WORK PROGRESS NOTE    SWer received voicemail message from patient's wife Jamie and call was returned.     SWer inquired with Jaime about intent to stay locally based on patient's conditioned to which she replied patient is headed toward being neutropenic. As SWer began presentation related to housing away from construction Jamie informed her she moved from the HOME apartments to the Foster City apartments this past week. Patient will join wife there when discharged. Plan is for patient and his wife to continue use of local Foster City apartment until patient has no medical reason to remain in the area.    SWer inquired as to other matters that she could assist with. Jamie had no additional concerns for now. SWer will visit with patient late today.    Mattie' Nogaye, LCSW      "

In [97]:
relevant_note_table.shape

(20340, 5)

In [98]:
print(f'Number of patient with relevant types of notes: {len(pd.unique(relevant_note_table["PAT_DEID"]))}')

Number of patient with relevant types of notes: 1795


In [99]:
relevant_note_table

,PAT_DEID,NOTE,EFFECTIVE_TIME,NOTE_DESC,WORD_COUNT
1,400384,Stanford Hospital and Clinics History and Phy...,2016-08-31 13:05:00,H&P,2031
2,400384,Stanford Hospital and Clinics History and Phy...,2016-08-10 17:31:00,H&P,2361
3,1618176,Stanford Hospital and Clinics Interventional ...,2013-03-11 08:57:00,H&P,471
4,2360321,Stanford Hospital and Clinics Interventional ...,2015-02-09 14:55:00,H&P,1484
5,2360321,Stanford Hospital and Clinics History and P...,2015-02-06 18:12:00,H&P,2070
...,...,...,...,...,...
35267,258815,Age: 54 Y Sex: male Diagnosis: Mantle cell ly...,2019-01-05 17:32:00,Progress Notes,737
35268,258815,Age: 54 Y Sex: male Diagnosis: Mantle Cell Ly...,2018-12-29 14:28:00,Progress Notes,154
35269,258815,Age: 54 Y Sex: male Diagnosis: Mantle cell ly...,2019-01-04 18:07:00,Progress Notes,980
35270,258815,"RN asked provider to see patient for ""not feel...",2019-01-17 08:44:00,Progress Notes,685


**Modify the cohort**

In [31]:
# Load relevant note table
sql_query = f""" SELECT * FROM {es}.note_by_type"""
note_by_type = (client.query(sql_query)).to_dataframe()

In [32]:
cohort_by_type = note_by_type[['PAT_DEID']]

In [33]:
cohort_by_type = pd.merge(cohort_by_type.drop_duplicates(subset="PAT_DEID"), cohort, how = 'left', on = ['PAT_DEID'])

In [34]:
print(f'Number of patient with relevant types of notes: {len(pd.unique(cohort_by_type["PAT_DEID"]))}')

Number of patient with relevant types of notes: 1724


In [35]:
# Save the cohort patient informations 
# Specify destination for storing dataframe
destination = f"som-nero-phi-boussard.ES_ACU_Oncology.cohort_by_type"

# Save file to Big Query
load_job = client.load_table_from_dataframe(dataframe = cohort_by_type,                                  
                                                   destination = destination,
                                                   job_config = job_config)

# Run the job:
load_job.result()

LoadJob<project=som-nero-phi-boussard, location=US, id=b556c52c-bafb-4922-a297-07aaff95662b>

### Select only the notes with relevant terms

In [267]:
# Initialize the vocabulary using the words used to define DX_NAME 
# Class_1 = Pain 
class_1 = ["PAIN","HEADACH","ALGIA","DYSURIA","DYNIA","ACHE","PRESSURE","DISCOMFORT","CRAMP", "TIGHTNESS","RADICULOPATHY","TENDERNES", "ABDOMINAL CRIS"]
# Class_2 = Nausea, Vomitting, Dehydration, diarrhea
dehydration = ["DEHY","HYPERNAT"]
diarrhea = ["DIAR"]
nausea = ["NAUS"]
emesis = ["EMES","VOMIT","GASTROENTER","GASTRIT"]
class_2 = dehydration + diarrhea + nausea + emesis 
# Class_3 = Anemia 
class_3 = ["ANEM","RED CELL"]
# Class_4 = Sepsis, neutropenia, fever, pneuomia 
fever =  ["FEVER","FEBRILE","HYPERTHERM","PYREX"]
neutropenia = ["NEUT","PANCY","AGRAN"]
sepsis = ["SEPS","INFLAM","CANDIDEMIA","SEPTIC"]
pneumonia = ["PNEUMO","CONSOLID"]
class_4 = fever + neutropenia + sepsis + pneumonia 
all_classes = class_1 + class_2 + class_3 + class_4

- Filter the notes & save relevant note table

In [37]:
relevant_note = note_by_type['NOTE'].str.upper().apply(lambda x: any(y in x for y in all_classes))
relevant_note_table = note_by_type.loc[relevant_note]
irrelevant_note_table = note_by_type.loc[~relevant_note]

In [39]:
note_removed_count = note_by_type.shape[0] - relevant_note_table.shape[0]
note_by_type.shape, relevant_note_table.shape, note_removed_count

((31291, 6), (18065, 6), 13226)

In [40]:
print("No of unique patients with at least one relevant note", len(pd.unique(relevant_note_table['PAT_DEID'])))

No of unique patients with at least one relevant note 1793


In [41]:
patient_removed_count = len(pd.unique(note_by_type['PAT_DEID'])) - len(pd.unique(relevant_note_table['PAT_DEID']))
patient_removed_count

22

In [185]:
patient_removed = pd.concat([pd.Series(pd.unique(relevant_note_table['PAT_DEID'])), pd.Series(pd.unique(note_by_type['PAT_DEID']))]).drop_duplicates(keep=False) 

In [189]:
patient_removed

109     2614543
181     1698842
362     1146163
499     1460038
502     1675846
537       38474
540     2876746
733      732261
756      154216
797      883310
985     1553033
1090    1294489
1251    1009842
1257      71347
1264    2205619
1330     384187
1373     655809
1528    4054999
1648    2935783
1654    2533352
1665    3277034
1738    2766580
dtype: Int64

In [43]:
# Save relevant note table
# Specify destination for storing dataframe
destination = f"som-nero-phi-boussard.ES_ACU_Oncology.relevant_note_table"

# Save file to Big Query
load_job = client.load_table_from_dataframe(dataframe = relevant_note_table,                                  
                                                   destination = destination,
                                                   job_config = job_config)

# Run the job:
load_job.result()

LoadJob<project=som-nero-phi-boussard, location=US, id=dff51e7a-1480-4326-8d79-5d62488b9158>

**(Optional:) Look at the removed patients**

- PAT_DEID = 1698842

In [119]:
df_notes = note_by_type.copy()

In [217]:
sub_note_table = df_notes[df_notes["PAT_DEID"] == 2766580].copy()
sub_note_table.shape

(2, 6)

In [218]:
sub_note_table

,PAT_DEID,NOTE,ENCOUNTER_DATE,EFFECTIVE_TIME,NOTE_DESC,WORD_COUNT
47552,2766580,Patient tolerated Rituxan infusion well. Vital...,2016-03-24,2016-03-24 16:21:00,Progress Notes,36
47553,2766580,Sent 4/1 cough assist order and Dr. So's note ...,2016-04-01,2016-04-01 13:46:00,Telephone Encounter,20


In [221]:
sub_note_table.NOTE.iloc[1]

"Sent 4/1 cough assist order and Dr. So's note from 2/25 to MBHC.    Fax: 408-934-9607  "

In [220]:
cohort[cohort["PAT_DEID"] == 2766580]

,PAT_DEID,CONTACT_DATE,CHE_TX_DATE
1148,2766580,2016-04-13,2016-03-24


Patient that should not be removed: 
- 38474: include "mouth soreness" or soreness or sore ("No ulcerations noted" -> include ulcerations?)
"lingering problems"?
- 732261: "abdomen is getting quite large and uncomfortable", "ascites worsens", "SOB"
- 154216: "Fatigue/weakness with walking"? Active Issues:   -Neuropathy in LLE r/t surgery per pt, uses cane  -Fatigue/weakness with walking 
- 1553033: "having hard time tolerating Prednisone"; "experienced bad anxiety and tachycardia"; "taking Ativan with Prednisone to help counter the side effects, and if still has terrible side effects to notify us"; "side effects of prednisone after 40 mg anxiety nervous"; "qod heart beating aster".
- 655809: "Treatment diagnosis:  Myositis"
- 2766580: "Sent 4/1 cough assist order and Dr. So's note from 2/25 to MBHC.    Fax: 408-934-9607  " -> include cough? 

"Patient is experiencing the following symptoms"? I don't think i should include the word symptom in my vocabulary"


I think when it start with this: "SOCIAL WORK ASSESSMENT" (progress notes) you can remove it

Removed patients that don't have the same number in note_table (all notes) and note_by_type (note of certain types): 
1698842
1460038
1675846
38474
2876746
732261
154216
883310
1553033
1294489
71347
2205619
655809
2935783
2533352
3277034

### Modify the cohort 

Need to modify the cohort -> check the stats; ask behzad for the proposal; run bert on small concatenated data on carina 

In [1]:
# Check patient with only one or 2 notes and check their notees

In [ ]:
## Check patients with min number and max number of notes

In [ ]:
#